In [1]:
import os
os.chdir("/home/sunflower124/25-26_internship/jaxlogit")

In [2]:
import pandas as pd
import jax
import numpy as np
import random

from jaxlogit.mixed_logit import MixedLogit, ConfigData
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("https://raw.github.com/arteagac/xlogit/master/examples/data/electricity_long.csv")

In [8]:
df = pd.read_csv("https://raw.github.com/arteagac/xlogit/master/examples/data/electricity_long.csv")
varnames = ['pf', 'cl', 'loc', 'wk', 'tod', 'seas']
n_draws = 5000

X = np.array(df[varnames])
y = np.array(df['choice'])

choice_situations = X.shape[0]
number_chosen = int(((choice_situations * 0.8) // 4) * 4)
chosen = random.sample(range(1, choice_situations), number_chosen)
selected = np.array([1 if i in chosen else 0 for i in range(0, choice_situations)]).astype(bool)

# X_train, X_test, y_train, y_test, ids_train, ids_test, alts_train, alts_test = train_test_split(X, y, np.array(df['chid']), np.array(df['alt']), test_size=0.201, random_state=1, stratify=None)

# X_train = X[selected, :]
# X_test = X[~selected, :]
# y_train = y[selected]
# y_test = y[~selected]

# ids_train = df['chid'][:number_chosen]
# ids_test = df['chid'][~selected]
# alts_train = df['alt'][selected]
# alts_test = df['alt'][~selected]

X_train = X[:number_chosen, :]
X_test = X[number_chosen:, :]
y_train = y[:number_chosen]
y_test = y[number_chosen:]

ids_train = df['chid'][:number_chosen]
ids_test = df['chid'][number_chosen:]
alts_train = df['alt'][:number_chosen]
alts_test = df['alt'][number_chosen:]

In [10]:
print(X_train.shape)
print(X_train)
print(y_train)
print(ids_train)
print(alts_train)

(13784, 6)
[[7 5 0 1 0 0]
 [9 1 1 0 0 0]
 [0 0 0 0 0 1]
 ...
 [0 0 0 0 1 0]
 [0 5 1 0 0 1]
 [9 5 0 1 0 0]]
[0 0 0 ... 0 1 0]
0           1
1           1
2           1
3           1
4           2
         ... 
13779    3445
13780    3446
13781    3446
13782    3446
13783    3446
Name: chid, Length: 13784, dtype: int64
0        1
1        2
2        3
3        4
4        1
        ..
13779    4
13780    1
13781    2
13782    3
13783    4
Name: alt, Length: 13784, dtype: int64


In [11]:
model = MixedLogit()

config = ConfigData(
    panels=None,
    n_draws=n_draws,
    skip_std_errs=True,  # skip standard errors to speed up the example
    batch_size=None,
    optim_method="L-BFGS-B",
)

model.fit(
    X=X_train,
    y=y_train,
    varnames=varnames,
    ids=ids_train,
    alts=alts_train,
    randvars={'pf': 'n', 'cl': 'n', 'loc': 'n', 'wk': 'n', 'tod': 'n', 'seas': 'n'},
    config=config
)
display(model.summary())

    Message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
    Iterations: 73
    Function evaluations: 112
Estimation time= 152.9 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
pf                     -0.7357440     1.0000000    -0.7357440         0.462    
cl                     -0.1604456     1.0000000    -0.1604456         0.873    
loc                     1.7625309     1.0000000     1.7625309        0.0781 .  
wk                      1.2794884     1.0000000     1.2794884         0.201    
tod                    -7.1351660     1.0000000    -7.1351662      1.17e-12 ***
seas                   -7.1921890     1.0000000    -7.1921892      7.78e-13 ***
sd.pf                  -8.5253730     1.0000000    -8.5253725      2.25e-17 ***
sd.cl                  -4.5321538     1.0000000    -4.5321536     

None

In [75]:
varnames = ['pf', 'cl', 'loc', 'wk', 'tod', 'seas']
varnames = ['pf', 'loc', 'wk', 'tod', 'seas']
X_restricted_train = X_train[:, [0,2,3,4,5]]

model = MixedLogit()

config = ConfigData(
    panels=None,
    n_draws=n_draws,
    skip_std_errs=True,  # skip standard errors to speed up the example
    batch_size=None,
    optim_method="L-BFGS-B",
)

model.fit(
    X=X_restricted_train,
    y=y_train,
    varnames=varnames,
    ids=ids_train,
    alts=alts_train,
    randvars={'pf': 'n', 'loc': 'n', 'wk': 'n', 'tod': 'n', 'seas': 'n'},
    config=config
)
display(model.summary())

ValueError: inconsistent alts values in long format, expected [1 2 3 ... 2 3 4], got [1 2 3 4]

In [ ]:
init_coeff = model.coeff_ if model.coeff_ is not None else np.array([-0.74000166, -0.16175459, 1.76466274, 1.28186378, -7.16826102, -7.24588124, -8.63775611, -4.68150211, 0.618132, -2.92170076, 1.90160474, 0.8248858])
config.init_coeff = init_coeff
X_restricted_test = X_test[:, [0,2,3,4,5]]
prob = model.predict(X_test, varnames, alts_test, ids_test, {'pf': 'n', 'cl': 'n', 'loc': 'n', 'wk': 'n', 'tod': 'n', 'seas': 'n'}, config)

ValueError: The length of init_coeff must be 11, but got 12.

In [ ]:
for i in range(prob.shape[0]):
    print(f"{prob[i]} : {y[i]}")

[0.14133394 0.6305913  0.1794783  0.0485965 ] : [0 1 0 0]
[0.2770637  0.26600832 0.10400182 0.35292614] : [0 0 0 1]
[0.32137123 0.10123405 0.4193965  0.15799826] : [1 0 0 0]
[0.32379466 0.1274967  0.30059433 0.2481143 ] : [0 0 1 0]
[0.16840048 0.24855547 0.47564062 0.10740346] : [0 1 0 0]
[0.35657126 0.46533334 0.08904771 0.08904771] : [0 1 0 0]
[0.16179667 0.15906623 0.21341582 0.46572125] : [0 0 0 1]
[0.21385881 0.38695666 0.0873779  0.31180656] : [0 1 0 0]
[0.35738966 0.23293349 0.252971   0.15670584] : [0 0 1 0]
[0.03680544 0.11955858 0.29052082 0.5531151 ] : [0 0 0 1]
[0.147255   0.09667623 0.0471715  0.7088973 ] : [0 0 0 1]
[0.23287283 0.35783115 0.15622401 0.25307202] : [0 0 0 1]
[0.26634192 0.08683304 0.29924303 0.347582  ] : [0 0 0 1]
[0.05848302 0.09552589 0.15584132 0.6901498 ] : [0 0 0 1]
[0.38682154 0.31181797 0.21401311 0.08734739] : [1 0 0 0]
[0.0422973  0.08787919 0.6356473  0.2341762 ] : [0 0 1 0]
[0.08731087 0.38665977 0.3121087  0.21392065] : [0 1 0 0]
[0.23267256 0.

In [46]:
y = np.reshape(y_test, (prob.shape[0], -1))

total_counted = 0
correct = 0
for i in range(prob.shape[0]):
    y_index = np.argmax(y[i])
    prob_index = np.argmax(prob[i])
    if prob_index == y_index:
        correct += 1
    total_counted += 1
print(f"percentage correct = {correct/total_counted}")

percentage correct = 0.45496535796766746
